In [1]:
import geopandas as gpd
import pandas as pd
from planet import api
from planet.api import downloader
from planet.api.downloader import create
import time
import os

In [2]:
item_type = 'PSScene'
asset_type = 'ortho_analytic_8b_sr'
backup_type = 'ortho_analytic_4b'
download_dir = './downloads/'

In [3]:
planet_api_key = input('PLANET API KEY')

PLANET API KEYPLAK72ac8990af84467483541a0e1fc60c61


In [4]:
downloaded_files = os.listdir(download_dir)
downloaded_IDs = []
for file in downloaded_files:
    if file[-3:] != 'tif':
        continue
    
    ID = file.split('_3B_Analytic')[0]
    downloaded_IDs.append(ID)

In [5]:
def activatePlanet(sceneID,item_type,asset_type):
    client = api.ClientV1(api_key=planet_api_key)
    assets = client.get_assets_by_id(item_type, sceneID).get()
    activation = client.activate(assets[asset_type])
    time.sleep(5)
    return asset_type

def downloadPlanetBatch(sceneIDs,item_type,asset_types,outDir):
    client = api.ClientV1(api_key=planet_api_key)
    completeFlag = 0

    while completeFlag == 0:
        sceneIDsPending = sceneIDs
        for sceneID in sceneIDsPending:
            asset_type = asset_types[sceneID]
            activeCheck = client.get_assets_by_id(item_type, sceneID).get()[asset_type]['status']
            if activeCheck == 'active':
                print('Downloading: '+sceneID)
                assets = client.get_assets_by_id(item_type, sceneID).get()
                callback = api.write_to_file(directory=outDir, callback= None, overwrite= True)
                body = client.download(assets[asset_type], callback=callback)
                body.wait()
                sceneIDs.remove(sceneID)

                print('\tFinished download')
                
        if len(sceneIDs) == 0:
            completeFlag = 1
        else:
            print('Waiting for activation of '+str(len(sceneIDs))+' planet images')
            time.sleep(60)

In [6]:
csvPath = './Collocation_HLS_Planet_2021.csv'
Collocation = pd.read_csv(csvPath, index_col=0)

In [7]:
chipIDs = Collocation.chip_ID
Strata = []
for ID in chipIDs:
    s = int(ID[0])
    Strata.append(s)

Collocation['Strata'] = Strata

In [8]:
Ranked = pd.DataFrame()
chipIDs_uniq = Collocation.chip_ID.unique()
for ID in chipIDs_uniq:
    search = Collocation[(Collocation.chip_ID == ID) & (Collocation.pl_Cloud_Cover == 0.0) & (Collocation.hls_Cloud_Cover <= 30.0)].sort_values(by=['hls_pl_AOI_intersect'],ascending=False)
    if len(search) > 0:
        top = search.iloc[0]
        Ranked = Ranked.append(top, ignore_index=True)
    else:
        search = Collocation[(Collocation.chip_ID == ID) & (Collocation.pl_Cloud_Cover == 0.0) & (Collocation.hls_Cloud_Cover <= 60.0)].sort_values(by=['hls_pl_AOI_intersect'],ascending=False)
        top = search.iloc[0]
        Ranked = Ranked.append(top, ignore_index=True)
Ranked.head()

,hls_Date,hls_ID,hls_Cloud_Cover,hls_Footprint,hls_AOI_intersect,hls_Link,hls_Thumbnail,pl_Date,pl_ID,pl_Cloud_Cover,pl_Footprint,pl_AOI_intersect,pl_Link,pl_Thumbnail,hls_pl_AOI_intersect,timeDelta_days,chip_ID,chip_rID,Strata
0,2021-09-28 22:56:33.623000+00:00,HLS.S30.T04WDB.2021271T225541.v2.0,4.0,"{'type': 'Polygon', 'coordinates': [[[-161.436...",100.00000,<Link rel=self target=https://cmr.earthdata.na...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2021-09-29 21:28:09.168336+00:00,20210929_212809_16_2442,0.0,"{'coordinates': [[[-160.0089598452984, 69.3047...",73.655138,https://api.planet.com/data/v1/item-types/PSSc...,https://tiles.planet.com/data/v1/item-types/PS...,73.655138,0.938606,3_38,99.0,3.0
1,2021-10-12 19:15:49.569000+00:00,HLS.S30.T12WVV.2021285T191341.v2.0,41.0,"{'type': 'Polygon', 'coordinates': [[[-113.259...",100.00000,<Link rel=self target=https://cmr.earthdata.na...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2021-10-12 18:45:51.578477+00:00,20211012_184551_1040,0.0,"{'coordinates': [[[-112.88824769135005, 67.212...",2.911913,https://api.planet.com/data/v1/item-types/PSSc...,https://tiles.planet.com/data/v1/item-types/PS...,2.911913,0.020810,4_27,114.0,4.0
2,2021-10-12 10:52:20.245000+00:00,HLS.S30.T33WXP.2021285T105011.v2.0,16.0,"{'type': 'Polygon', 'coordinates': [[[17.29852...",29.50247,<Link rel=self target=https://cmr.earthdata.na...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2021-10-13 09:25:04.083607+00:00,20211013_092504_08_2442,0.0,"{'coordinates': [[[17.407137633889764, 66.1944...",63.344900,https://api.planet.com/data/v1/item-types/PSSc...,https://tiles.planet.com/data/v1/item-types/PS...,21.366709,0.939396,4_43,20.0,4.0
3,2021-09-25 07:41:15.483000+00:00,HLS.S30.T40WFU.2021268T073641.v2.0,1.0,"{'type': 'Polygon', 'coordinates': [[[61.56600...",95.95395,<Link rel=self target=https://cmr.earthdata.na...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2021-09-26 07:28:58.882736+00:00,20210926_072858_88_2416,0.0,"{'coordinates': [[[59.01856566651295, 65.96493...",57.756871,https://api.planet.com/data/v1/item-types/PSSc...,https://tiles.planet.com/data/v1/item-types/PS...,55.331216,0.991475,3_1,112.0,3.0
4,2021-09-01 18:06:29.196000+00:00,HLS.S30.T15VUK.2021244T175911.v2.0,1.0,"{'type': 'Polygon', 'coordinates': [[[-96.8314...",100.00000,<Link rel=self target=https://cmr.earthdata.na...,https://data.lpdaac.earthdatacloud.nasa.gov/lp...,2021-09-01 17:52:05.711814+00:00,20210901_175205_71_2426,0.0,"{'coordinates': [[[-96.94381411041722, 62.8887...",96.416886,https://api.planet.com/data/v1/item-types/PSSc...,https://tiles.planet.com/data/v1/item-types/PS...,96.416886,0.009994,4_34,115.0,4.0


In [9]:
IDs = Ranked.pl_ID.tolist()
print(IDs)

['20210929_212809_16_2442', '20211012_184551_1040', '20211013_092504_08_2442', '20210926_072858_88_2416', '20210901_175205_71_2426', '20211030_014006_52_2431', '20211014_152910_49_2459', '20210918_042559_64_2419', '20211007_081605_77_245f', '20210921_182414_03_241d', '20211012_173555_20_2276', '20210914_103644_25_2413', '20210925_191708_94_2408', '20211008_073033_21_245c', '20211022_031446_34_2431', '20211006_164259_80_2460', '20211002_084046_69_245c', '20211029_070331_23_2428', '20211027_151445_21_2231', '20211009_070145_70_2451', '20211013_073019_14_2414', '20211010_080351_47_2458', '20210903_152641_60_105c', '20211008_060306_49_2439', '20211015_114738_1105', '20211028_045455_02_2459', '20211008_063123_28_245f', '20210917_140704_93_2262', '20211028_151242_66_2428', '20211026_025125_77_240c', '20211008_054349_78_241d', '20211003_075037_07_2458', '20210912_034049_22_2421', '20211019_180019_07_2453', '20210906_030035_64_2440', '20211001_071025_45_2435', '20211028_055109_45_245f', '20211

In [10]:
for ID in downloaded_IDs:
    if ID in IDs:
        IDs.remove(ID)
print(len(IDs))

130


In [11]:
IDs = ['20211012_184551_1040']

In [13]:
asset_types = {}
for ID in IDs:
    try:
        aType = activatePlanet(ID,item_type,asset_type)
        print('Activated ID:'+ID)
    except:
        try:
            aType = activatePlanet(ID,item_type,backup_type)
            print('Activated ID (backup type):'+ID)
        except:
            print('Failed to activate ID:'+ID)
    asset_types[ID] = aType

Activated ID (backup type):20211012_184551_1040


In [ ]:
copyIDs = IDs

In [ ]:
downloadPlanetBatch(IDs,item_type,asset_types,download_dir)